In [1]:
!nvidia-smi

Sat Feb 17 11:08:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000001:00:00.0 Off |                    0 |
| N/A   66C    P0             405W / 400W |  45149MiB / 81920MiB |    100%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [3]:
from modeling_combine import MM_LLMs, MM_LLMs_Config
from transformers import AutoModelForCausalLM, CLIPProcessor, CLIPModel,AutoModel, AutoTokenizer, AutoProcessor,AutoConfig,CLIPConfig, LlamaConfig, WhisperConfig, WhisperModel, LlamaModel, LlamaTokenizer
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torch
import numpy as np
from torch import nn
from streaming import LocalDataset
from typing import List

In [4]:
MM_LLMs.register_for_auto_class()
MM_LLMs_Config.register_for_auto_class()

In [5]:
from transformers.trainer_utils import get_last_checkpoint

latest = get_last_checkpoint('multimodal-mistral')
latest

'multimodal-mistral/checkpoint-5100'

In [6]:
model = MM_LLMs.from_pretrained(
    latest,flash_attention = True, dtype = torch.bfloat16, torch_dtype = torch.bfloat16
)

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
_ = model.cuda()

In [8]:
image_processor = AutoProcessor.from_pretrained('google/siglip-base-patch16-384')
audio_processor = AutoProcessor.from_pretrained('mesolitica/malaysian-whisper-small')
tokenizer = AutoTokenizer.from_pretrained(latest)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
default_height = image_processor.image_processor.size['height']

In [10]:
from PIL import Image
import librosa

In [11]:
def prepare_dataset(messages, images: List[str] = None, audio: List[str] = None, sr = 16000):
    
    if images is not None:
        images = [Image.open(f).convert('RGB') for f in images]
        image_output = image_processor(images=images, return_tensors='pt')['pixel_values']
    else:
        image_output = None
        
    if audio is not None:
        audio = [librosa.load(f, sr=sr)[0] for f in audio]
        audio_features = audio_processor(audio, sampling_rate=sr, return_tensors='pt',)['input_features']
    else:
        audio_features = None
    
    prompt = tokenizer.apply_chat_template(messages, tokenize = False)
    outputs = tokenizer(
                    prompt,
                    return_tensors='pt',
                    return_overflowing_tokens=False,
                    return_length=False
    )

    outputs['images'] = image_output
    outputs['audios'] = audio_features
    
    image_token = tokenizer.convert_tokens_to_ids('<image>')
    audio_token = tokenizer.convert_tokens_to_ids('<audio>')
    
    if image_output is not None:
        len_image = len(image_output)
    else:
        len_image = 0
        
    if audio_features is not None:
        len_audio = len(audio_features)
    else:
        len_audio = 0
        
    outputs['image_index'] = torch.tensor([0] * len_image)
    outputs['image_starts'] = torch.tensor([image_token] * (len_image + 1))
    outputs['audio_index'] = torch.tensor([0] * len_audio)
    outputs['audio_starts'] = torch.tensor([audio_token] * (len_audio + 1))
        
    where_is = torch.where((outputs['input_ids'] == image_token) | (outputs['input_ids'] == audio_token))
    ls = []
    for i in range(len(where_is[0])):
        b, k = where_is[0][i], where_is[1][i]
        l = int(outputs['input_ids'][b, k])
        ls.append(l)

    ls = torch.tensor(ls)
    outputs['where_is_b'] = where_is[0]
    outputs['where_is_k'] = where_is[1]
    outputs['ls'] = ls
        
    return outputs

In [12]:
# !wget https://cdn.beautifulnara.net/wp-content/uploads/2017/12/10201620/Persian-cat-breed.jpg
# !wget https://www.jocooks.com/wp-content/uploads/2023/09/nasi-goreng-1-23.jpg

In [40]:
ls = []
with open('malaysian-youtube-audio-instructions/mixtral-audio-instruction.jsonl') as fopen:
    for l in fopen:
        ls.append(json.loads(l))
        if len(ls) > 100:
            break

In [31]:
!ls malaysian-youtube-audio-instructions/filter-audio/1-0-0.mp3

malaysian-youtube-audio-instructions/filter-audio/1-0-0.mp3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [48]:
!cp malaysian-youtube-audio-instructions/filter-audio/1-0-0.mp3 1.mp3
!cp malaysian-youtube-audio-instructions/filter-audio/1-2470-31.mp3 2.mp3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
audio_text1 = 'anda tahu keuntungan boleh lebih tinggi daripada keuntungan kewangan rumah maka saya tidak akan mencari dalam akaun saya akan mencari ke dalam ethereum atau beberapa crypto punks bergantung pada faktor risiko anda kerana rumah kajang dihantar tidak mengganggu dsr saya sejauh ini jadi sekarang apa posisi saya untuk mendapatkan kewangan ketiga jadi mungkin setelah melihat sekeliling saya menemui seorang penjual yang dapat menutupi perhubungan tetapi bank hanya menerima 70% dari itu saya boleh membayar perbezaan dengan menggunakan wang ini kerana sekali lagi ia menyusahkan saya dan aset tetapi jika anda tidak selesa dengan mencari'

In [50]:
audio_text2 = 'Sekarang Adria nak tunjukkan dia punya kotak Kotak dia simple, dia transparent Vib Max dekat sini, root beer Dia punya flavour punya nama Dekat atas dia tulis root beer float Bagi korang senang sikit nak tengok Tepi sini 12,000 puff Lepas tu dekat sebelah sini dia ada tulis lah dekat sini 22ml untuk 12,000 puff lah Rechargeable type C, 600mAh Adjustable airflow So dekat atas ada satu seal Korang just bukakan Untuk dia punya silicon actually agak besar Masa korang ambil keluar device yang baru kan Dia ada 2 sticker, 1 silicon'

In [13]:
test_image = 'translated-LLaVA-Instruct-150K/filtered-llava-images/000000033471.jpg'
test_image2 = 'Persian-cat-breed.jpg'
test_image3 = 'abang-gay.png'
test_image4 = 'nasi-goreng-1-23.jpg'
images = [
    test_image,
    test_image2,
    test_image3,
    test_image4
]
audio = 'test.mp3'

In [14]:
for img in images:
    messages = [
        {'role': 'user', 'content': '<image> </image> ini gambar apa'},
    ]
    outputs = prepare_dataset(messages, images = [img])
    if outputs['images'] is not None:
        outputs['images'] = outputs['images'].type(model.dtype)
    if outputs['audios'] is not None:
        outputs['audios'] = outputs['audios'].type(model.dtype)
    for k in outputs.keys():
        if outputs[k] is not None:
            outputs[k] = outputs[k].cuda()

    with torch.no_grad():
        model_inputs = model.prepare_inputs_for_generation(**outputs, inference = True)

    r = model_inputs.pop('input_ids', None)
    generate_kwargs = dict(
        model_inputs,
        max_new_tokens=300,
        top_p=0.95,
        top_k=50,
        temperature=0.1,
        do_sample=True,
        num_beams=1,
    )

    r = model.llm.generate(**generate_kwargs)
    print(img, tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


translated-LLaVA-Instruct-150K/filtered-llava-images/000000033471.jpg <s>Imej itu menunjukkan bas dua tingkat putih yang diletakkan di jalan bandar. Bas itu mempunyai iklan di sisinya, mungkin untuk syarikat pengangkutan atau perniagaan lain. Terdapat beberapa orang di tempat kejadian, berjalan di sepanjang jalan dan berinteraksi antara satu sama lain.

Terdapat beberapa kereta yang diletakkan di sepanjang jalan, termasuk satu di sebelah kiri bas dan satu lagi di sebelah kanan. Selain itu, terdapat dua motosikal yang diletakkan di sebelah kanan imej.</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Persian-cat-breed.jpg <s>Imej ini mempunyai kucing putih yang terbaring di atas sofa hitam.</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


abang-gay.png <s>Ini adalah gambar seorang lelaki muda yang tersenyum, memakai baju polo hitam dan memegang beg galas. Dia kelihatan berada di luar, mungkin di sebuah bangunan.</s>
nasi-goreng-1-23.jpg <s>Imej ini menampilkan mangkuk putih yang diisi dengan nasi dan pelbagai sayur-sayuran, termasuk lobak merah, brokoli dan kacang. Mangkuk itu diletakkan di atas meja, dan terdapat sudu hitam diletakkan di sebelahnya.</s>


In [15]:
messages = [
    {'role': 'user', 'content': '<image> </image> <image> </image> apa kaitan 2 gambar ni'},
]
outputs = prepare_dataset(messages, images = [test_image, test_image2])
if outputs['images'] is not None:
    outputs['images'] = outputs['images'].type(model.dtype)
if outputs['audios'] is not None:
    outputs['audios'] = outputs['audios'].type(model.dtype)
for k in outputs.keys():
    if outputs[k] is not None:
        outputs[k] = outputs[k].cuda()

with torch.no_grad():
    model_inputs = model.prepare_inputs_for_generation(**outputs, inference = True)
    
r = model_inputs.pop('input_ids', None)
generate_kwargs = dict(
    model_inputs,
    max_new_tokens=300,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
)

r = model.llm.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Tiada kaitan langsung antara dua gambar itu. Gambar pertama menunjukkan bas pelancongan dengan iklan di sisinya, manakala gambar kedua menunjukkan seekor kucing putih yang comel terbaring di atas sofa. Kedua-dua imej adalah berasingan dan tidak berkaitan antara satu sama lain.</s>


In [24]:
messages = [
    {'role': 'user', 'content': '<image> </image> <image> </image> What is related between image 1 and image 2'},
]
outputs = prepare_dataset(messages, images = [test_image, test_image2])
if outputs['images'] is not None:
    outputs['images'] = outputs['images'].type(model.dtype)
if outputs['audios'] is not None:
    outputs['audios'] = outputs['audios'].type(model.dtype)
for k in outputs.keys():
    if outputs[k] is not None:
        outputs[k] = outputs[k].cuda()

with torch.no_grad():
    model_inputs = model.prepare_inputs_for_generation(**outputs, inference = True)
    
r = model_inputs.pop('input_ids', None)
generate_kwargs = dict(
    model_inputs,
    max_new_tokens=300,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
)

r = model.llm.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>There is no direct relationship between image 1 and image 2. The first image shows a tour bus driving down a street, while the second image shows a white cat lying on a black couch. Both are unrelated to each other.</s>


In [16]:
messages = [
    {'role': 'user', 'content': '<audio> </audio> apa isu audio ni'},
]
outputs = prepare_dataset(messages, audio = [audio])
if outputs['images'] is not None:
    outputs['images'] = outputs['images'].type(model.dtype)
if outputs['audios'] is not None:
    outputs['audios'] = outputs['audios'].type(model.dtype)
for k in outputs.keys():
    if outputs[k] is not None:
        outputs[k] = outputs[k].cuda()

with torch.no_grad():
    model_inputs = model.prepare_inputs_for_generation(**outputs, inference = True)
    
r = model_inputs.pop('input_ids', None)
generate_kwargs = dict(
    model_inputs,
    max_new_tokens=300,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
)

r = model.llm.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Isu yang dibangkitkan dalam audio ini ialah kekurangan pemahaman dan kesedaran tentang faedah menggunakan e-dompet di kalangan orang ramai, terutamanya di Malaysia. Penceramah menyatakan kekecewaan mereka kerana walaupun kerajaan memberi insentif kepada orang ramai untuk menggunakan e-dompet, ramai orang tidak memahami objektif di sebaliknya. Mereka berhujah bahawa insentif kewangan tidak mencukupi untuk menggalakkan orang ramai menggunakan e-dompet, kerana mereka mungkin tidak memahami cara menggunakannya atau mungkin tidak melihat nilai dalam menggunakannya.

Penceramah juga menyebut isu khusus di Malaysia, di mana orang ramai mungkin tidak dapat menggunakan e-dompet mereka kerana kekurangan infrastruktur yang betul. Sebagai contoh, jika seseorang ingin menggunakan e-


In [17]:
messages = [
    {'role': 'user', 'content': '<image> </image> <audio> </audio> apa kaitan gambar dan audio ni'},
]
outputs = prepare_dataset(messages, images = [test_image], audio = [audio])
if outputs['images'] is not None:
    outputs['images'] = outputs['images'].type(model.dtype)
if outputs['audios'] is not None:
    outputs['audios'] = outputs['audios'].type(model.dtype)
for k in outputs.keys():
    if outputs[k] is not None:
        outputs[k] = outputs[k].cuda()

with torch.no_grad():
    model_inputs = model.prepare_inputs_for_generation(**outputs, inference = True)
    
r = model_inputs.pop('input_ids', None)
generate_kwargs = dict(
    model_inputs,
    max_new_tokens=300,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
)

r = model.llm.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>The audio is discussing the issue of people not understanding the objective behind encouraging the use of e-wallets in Malaysia. The speaker mentions that if people were given RM30 or RM50, they would not use e-wallets because they would not understand the purpose of it. The speaker also mentions that there are many problems in Malaysia, such as the difficulty of using e-wallets in certain places, and the lack of a proper ecosystem.

The picture shows a bus with a "Pay by E-wallet" sign on it. This picture is not directly related to the audio as it does not provide any context or information about the audio's topic. However, the picture could be used to illustrate the concept of using e-wallets for paying for things, which is mentioned in the audio.</s>


In [52]:
messages = [
    {'role': 'user', 'content': '<image> </image> <audio> </audio>  What is related between image 1 and audio 1?'},
]
outputs = prepare_dataset(messages, images = [test_image2], audio = ['1.mp3'])
if outputs['images'] is not None:
    outputs['images'] = outputs['images'].type(model.dtype)
if outputs['audios'] is not None:
    outputs['audios'] = outputs['audios'].type(model.dtype)
for k in outputs.keys():
    if outputs[k] is not None:
        outputs[k] = outputs[k].cuda()

with torch.no_grad():
    model_inputs = model.prepare_inputs_for_generation(**outputs, inference = True)
    
r = model_inputs.pop('input_ids', None)
generate_kwargs = dict(
    model_inputs,
    max_new_tokens=300,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
)

r = model.llm.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>There is no direct relationship between image 1 and audio 1. The image shows a white cat lying on a black couch, while the audio is about a person discussing their decision to invest their money in cryptocurrencies instead of a housing loan.</s>


In [54]:
messages = [
    {'role': 'user', 'content': '<audio> </audio> <audio> </audio> What is related for both audio?'},
]
outputs = prepare_dataset(messages, audio = ['1.mp3', '2.mp3'])
if outputs['images'] is not None:
    outputs['images'] = outputs['images'].type(model.dtype)
if outputs['audios'] is not None:
    outputs['audios'] = outputs['audios'].type(model.dtype)
for k in outputs.keys():
    if outputs[k] is not None:
        outputs[k] = outputs[k].cuda()

with torch.no_grad():
    model_inputs = model.prepare_inputs_for_generation(**outputs, inference = True)
    
r = model_inputs.pop('input_ids', None)
generate_kwargs = dict(
    model_inputs,
    max_new_tokens=300,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
)

r = model.llm.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Both audio clips are discussing the topic of buying a product, possibly a vaping device, and providing information about its features and specifications. In Audio 1, the speaker is discussing the decision to invest in cryptocurrencies instead of a house loan due to the potential higher returns. In Audio 2, the speaker is describing the features of a vaping device, including its transparency, simple design, and adjustable airflow. The speaker also mentions the device's charging capabilities and the amount of puffs it can provide.

The connection between the two audio clips is that they both discuss the importance of making informed decisions when purchasing a product. In Audio 1, the speaker is discussing the decision to invest in cryptocurrencies, while in Audio 2, the speaker is discussing the features and specifications of a vaping device. Both clips emphasize the importance of understanding the product's features and specifications before making a purchase decision.</s>


In [18]:
# Image.open('abang-gay.png')

In [19]:
model.push_to_hub('malaysian-mistral-mmmmodal', organization='mesolitica', safe_serialization=True)

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/utils/hub.py:667: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/822M [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-mistral-mmmmodal/commit/6f9a13b25d465588e464dbf45d8972896efc6ec9', commit_message='Upload MM_LLMs', commit_description='', oid='6f9a13b25d465588e464dbf45d8972896efc6ec9', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
image_processor.push_to_hub('malaysian-mistral-mmmmodal', organization='mesolitica', safe_serialization=True)

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/utils/hub.py:667: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-mistral-mmmmodal/commit/7521f7511d187bd1d2531a5679493c52c815e9ba', commit_message='Upload processor', commit_description='', oid='7521f7511d187bd1d2531a5679493c52c815e9ba', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
audio_processor.push_to_hub('malaysian-mistral-mmmmodal', organization='mesolitica', safe_serialization=True)

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-mistral-mmmmodal/commit/260f3862b53150d91e274b8bd8ea784a1efb405e', commit_message='Upload processor', commit_description='', oid='260f3862b53150d91e274b8bd8ea784a1efb405e', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
tokenizer.push_to_hub('malaysian-mistral-mmmmodal', organization='mesolitica', safe_serialization=True)

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-mistral-mmmmodal/commit/9a06d2a355873ea5773dfd9eaa6a30f20089c2db', commit_message='Upload tokenizer', commit_description='', oid='9a06d2a355873ea5773dfd9eaa6a30f20089c2db', pr_url=None, pr_revision=None, pr_num=None)